Простой пример: Обход списка слов (GoF-стиль)
Этот пример демонстрирует явное создание классов Итератора и Агрегата для обхода простого списка.

In [1]:
from abc import ABC, abstractmethod
from typing import List, Any

# 1. Iterator Interface (Интерфейс Итератора)
class Iterator(ABC):
    @abstractmethod
    def has_next(self) -> bool:
        """Проверяет, есть ли еще элементы для обхода."""
        pass

    @abstractmethod
    def next(self) -> Any:
        """Возвращает следующий элемент."""
        pass

# 2. Aggregate Interface (Интерфейс Агрегата/Коллекции)
class Aggregate(ABC):
    @abstractmethod
    def create_iterator(self) -> Iterator:
        """Создает итератор для этой коллекции."""
        pass

# 3. Concrete Iterator (Конкретный Итератор для списка)
class WordListIterator(Iterator):
    """Итератор для обхода списка слов вперед."""
    def __init__(self, collection: List[str]):
        self._collection = collection
        self._position = 0 # Текущая позиция в коллекции

    def has_next(self) -> bool:
        """Проверяем, не вышли ли мы за пределы списка."""
        return self._position < len(self._collection)

    def next(self) -> Any:
        """Возвращает текущий элемент и сдвигает позицию."""
        if not self.has_next():
            # В классическом варианте можно было бы вернуть None или бросить исключение
            # Python обычно бросает StopIteration, но здесь для простоты вернем None
            print("WordListIterator: No more elements.")
            return None
        value = self._collection[self._position]
        print(f"WordListIterator: Returning '{value}' at position {self._position}")
        self._position += 1
        return value

# 4. Concrete Aggregate (Конкретный Агрегат - список слов)
class WordList(Aggregate):
    """Коллекция слов, реализованная как список."""
    def __init__(self):
        self._words: List[str] = []

    def add_word(self, word: str):
        print(f"WordList: Adding word '{word}'")
        self._words.append(word)

    def get_words(self) -> List[str]:
        """Метод для получения самих данных (не часть паттерна Итератор)."""
        return self._words

    def create_iterator(self) -> Iterator:
        """Создает итератор для обхода нашего списка слов."""
        print("WordList: Creating forward iterator.")
        return WordListIterator(self._words)

# 5. Client Code (Клиентский Код)
if __name__ == "__main__":
    word_collection = WordList()
    word_collection.add_word("Hello")
    word_collection.add_word("Iterator")
    word_collection.add_word("Pattern")

    print("\nClient: Getting iterator and traversing forward...")
    iterator = word_collection.create_iterator()

    # Используем цикл while с has_next() и next()
    while iterator.has_next():
        word = iterator.next()
        if word is not None:
            print(f"Client: Processing word - {word.upper()}")

    print("\nClient: Traversal finished.")

# Вывод:
# WordList: Adding word 'Hello'
# WordList: Adding word 'Iterator'
# WordList: Adding word 'Pattern'
#
# Client: Getting iterator and traversing forward...
# WordList: Creating forward iterator.
# WordListIterator: Returning 'Hello' at position 0
# Client: Processing word - HELLO
# WordListIterator: Returning 'Iterator' at position 1
# Client: Processing word - ITERATOR
# WordListIterator: Returning 'Pattern' at position 2
# Client: Processing word - PATTERN
# WordListIterator: No more elements.
#
# Client: Traversal finished.

WordList: Adding word 'Hello'
WordList: Adding word 'Iterator'
WordList: Adding word 'Pattern'

Client: Getting iterator and traversing forward...
WordList: Creating forward iterator.
WordListIterator: Returning 'Hello' at position 0
Client: Processing word - HELLO
WordListIterator: Returning 'Iterator' at position 1
Client: Processing word - ITERATOR
WordListIterator: Returning 'Pattern' at position 2
Client: Processing word - PATTERN

Client: Traversal finished.


Сложный пример: Обход Дерева (Pythonic-стиль с генератором)
Этот пример показывает, как реализовать обход бинарного дерева (in-order traversal) с использованием встроенных механизмов Python (__iter__ и yield), что гораздо проще и естественнее для языка.

In [2]:
from typing import Any, Optional, Iterator as TypingIterator # Переименовываем, чтобы не конфликтовать с нашим простым примером

# Узел дерева
class TreeNode:
    def __init__(self, value: Any, left: Optional['TreeNode'] = None, right: Optional['TreeNode'] = None):
        self.value = value
        self.left = left
        self.right = right

    def __str__(self):
        return f"Node({self.value})"

# Aggregate (Коллекция - бинарное дерево)
class BinaryTree:
    """Представляет бинарное дерево."""
    def __init__(self, root: Optional[TreeNode] = None):
        self.root = root
        print("BinaryTree initialized.")

    # Реализация протокола Итератора Python: метод __iter__
    # Он возвращает объект-итератор (в данном случае, сам генератор)
    def __iter__(self) -> TypingIterator[Any]:
        """Возвращает итератор для in-order обхода (левый-корень-правый)."""
        print("BinaryTree: __iter__ called, starting in-order traversal...")
        return self._in_order_traversal(self.root)

    # Генератор для in-order обхода
    def _in_order_traversal(self, node: Optional[TreeNode]) -> TypingIterator[Any]:
        """Рекурсивный генератор для обхода."""
        if node is not None:
            # 1. Рекурсивно обходим левое поддерево
            # yield from делегирует итерацию другому генератору
            yield from self._in_order_traversal(node.left)

            # 2. Посещаем (возвращаем) текущий узел
            print(f"  (Yielding value: {node.value})")
            yield node.value # Возвращаем значение узла

            # 3. Рекурсивно обходим правое поддерево
            yield from self._in_order_traversal(node.right)

    # Можно добавить другие методы для создания других итераторов
    def pre_order_iterator(self) -> TypingIterator[Any]:
         """Возвращает итератор для pre-order обхода (корень-левый-правый)."""
         print("BinaryTree: pre_order_iterator called...")
         return self._pre_order_traversal(self.root)

    def _pre_order_traversal(self, node: Optional[TreeNode]) -> TypingIterator[Any]:
         if node is not None:
             print(f"  (Pre-order yielding value: {node.value})")
             yield node.value
             yield from self._pre_order_traversal(node.left)
             yield from self._pre_order_traversal(node.right)


# Client Code (Клиентский Код)
if __name__ == "__main__":
    # Строим дерево:
    #      4
    #     / \
    #    2   5
    #   / \
    #  1   3
    root = TreeNode(4,
                    left=TreeNode(2,
                                  left=TreeNode(1),
                                  right=TreeNode(3)),
                    right=TreeNode(5))
    tree = BinaryTree(root)

    print("\nClient: Iterating using Python's 'for' loop (uses __iter__ for in-order)...")
    # Клиент просто использует стандартный цикл for!
    for value in tree: # Python неявно вызывает iter(tree), который вызывает tree.__iter__()
        print(f"Client: Processing value - {value * 10}")

    print("\nClient: Iterating using pre-order iterator explicitly...")
    pre_order_iter = tree.pre_order_iterator()
    try:
        while True:
            value = next(pre_order_iter) # Явно вызываем next()
            print(f"Client: Processing pre-order value - {value}")
    except StopIteration:
        print("Client: Pre-order traversal finished.")


# Вывод:
# BinaryTree initialized.
#
# Client: Iterating using Python's 'for' loop (uses __iter__ for in-order)...
# BinaryTree: __iter__ called, starting in-order traversal...
#   (Yielding value: 1)
# Client: Processing value - 10
#   (Yielding value: 2)
# Client: Processing value - 20
#   (Yielding value: 3)
# Client: Processing value - 30
#   (Yielding value: 4)
# Client: Processing value - 40
#   (Yielding value: 5)
# Client: Processing value - 50
#
# Client: Iterating using pre-order iterator explicitly...
# BinaryTree: pre_order_iterator called...
#   (Pre-order yielding value: 4)
# Client: Processing pre-order value - 4
#   (Pre-order yielding value: 2)
# Client: Processing pre-order value - 2
#   (Pre-order yielding value: 1)
# Client: Processing pre-order value - 1
#   (Pre-order yielding value: 3)
# Client: Processing pre-order value - 3
#   (Pre-order yielding value: 5)
# Client: Processing pre-order value - 5
# Client: Pre-order traversal finished.

BinaryTree initialized.

Client: Iterating using Python's 'for' loop (uses __iter__ for in-order)...
BinaryTree: __iter__ called, starting in-order traversal...
  (Yielding value: 1)
Client: Processing value - 10
  (Yielding value: 2)
Client: Processing value - 20
  (Yielding value: 3)
Client: Processing value - 30
  (Yielding value: 4)
Client: Processing value - 40
  (Yielding value: 5)
Client: Processing value - 50

Client: Iterating using pre-order iterator explicitly...
BinaryTree: pre_order_iterator called...
  (Pre-order yielding value: 4)
Client: Processing pre-order value - 4
  (Pre-order yielding value: 2)
Client: Processing pre-order value - 2
  (Pre-order yielding value: 1)
Client: Processing pre-order value - 1
  (Pre-order yielding value: 3)
Client: Processing pre-order value - 3
  (Pre-order yielding value: 5)
Client: Processing pre-order value - 5
Client: Pre-order traversal finished.
